In [1]:
import yaml
from lvmtipo.scraper import Scraper

config = """
lvm.sci.pwi:
    ra_j2000_hours: ra_j2000_h
    dec_j2000_degs: dec_j2000_d
    altitude_degs: altitude_d
    azimuth_degs: azimuth_d

lvm.sci.foc:
    Position: foc_dt

lvm.sci.km:
    Position: km_d
    SkyPA: sky_d

lvm.sci.agcam:
    east.temperature: east.temp
    east.filename: east.file
    west.temperature: west.temp
    west.filename: west.file
    center.temperature: center.temp
    center.filename: center.file
"""

scraper = await Scraper(yaml.safe_load(config)).start()


In [2]:
from lvmtipo.actors import lvm

await lvm.sci.start()

await lvm.sci.km.status()
await lvm.sci.foc.status()
await lvm.sci.pwi.status()
await lvm.sci.agc.status()

for k, v in scraper.scraper_store.items():
    print(f"{v[1]} {k:>14}: {v[0]:.4f}")

2023-02-20 01:01:36     ra_j2000_h: 5.9780
2023-02-20 01:01:36    dec_j2000_d: -30.8853
2023-02-20 01:01:36     altitude_d: 85.6549
2023-02-20 01:01:36      azimuth_d: 243.3176
2023-02-20 01:00:27           km_d: -135.0000
2023-02-20 01:00:27          sky_d: -270.0000
2023-02-20 01:00:27         foc_dt: 39.0000
2023-02-20 01:00:27      east.temp: 46.3750
2023-02-20 01:00:27    center.temp: 48.1250


In [35]:
for k, v in scraper.scraper_store.items():
    print(f"{v[1]} {k:>14}: {v[0]:.4f}")

2023-02-09 14:07:26           km_d: 62.5444
2023-02-09 14:07:26          sky_d: 125.0889
2023-02-09 14:07:25     ra_j2000_h: 18.4031
2023-02-09 14:07:25    dec_j2000_d: -34.3855
2023-02-09 14:07:25     altitude_d: 83.5847
2023-02-09 14:07:25      azimuth_d: 212.2740
2023-02-09 14:06:39         foc_dt: 70.0000
2023-02-09 14:06:39      east.temp: 25.0000
2023-02-09 14:06:39    center.temp: 25.0000
2023-02-09 14:06:39      west.temp: 25.0000
